In [3]:
##We load the packages used at the work
##We are going to use as rule pd for Pandas
##If you are going to use Matplotlib, Numpy or
##other, you should search how to name them 
## as rule.
##For keep the order you can use prettier
## in vscode

##Pandas is similar to tidyverse
import pandas as pd 
##The next package is used to format tables
##In the next lines you can watch how to use it
from tabulate import tabulate



##La tabla que vamos a emplear es esta, las demás han
##sido eliminadas
file="SouthGermanCredit.asc"
data=pd.read_csv(file," ")

##We are going to change the name of the variables 
##because in German is difficult to us work.
names={'laufkont':'Status',
        'laufzeit':'Duracion',
        'moral':'Historial_Crediticio',
        'verw':'Proposito',
        'hoehe':'Monto',
        'sparkont':'Ahorros',
        'beseit':'Tiempo_trabajando',
        'rate':'Porcentaje_salario',
        'famges':'Estado_civil_sexo',
        'buerger':'Otros_deudores',
        'wohnzeit':'Residencia',
        'verm':'Propiedad_valiosa',
        'alter':'Edad',
        'other_installment_plans':'Otra_deuda',
        'wohn':'Residencia',
        'bishkred':'Creditos_banco',
        'beruf':'Empleo',
        'pers':'Personas_dependientes',
        'telef':'Telefono',
        'gastarb':'Extranjero',
        'kredit':'Default'}


data.rename(columns=names,inplace=True)
print(data.head(5))
#print(tabulate(data.head(5),headers='keys',tablefmt='latex_longtable'))


   Status  Duracion  Historial_Crediticio  Proposito  Monto  Ahorros  beszeit  \
0       1        18                     4          2   1049        1        2   
1       1         9                     4          0   2799        1        3   
2       2        12                     2          9    841        2        4   
3       1        12                     4          0   2122        1        3   
4       1        12                     4          0   2171        1        3   

   Porcentaje_salario  Estado_civil_sexo  buerge  ...  Propiedad_valiosa  \
0                   4                  2       1  ...                  2   
1                   2                  3       1  ...                  1   
2                   2                  2       1  ...                  1   
3                   3                  3       1  ...                  1   
4                   4                  3       1  ...                  2   

   Edad  weitkred  Residencia  Creditos_banco  Empleo  P

c:\Users\saac9\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3397: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
##We are going to construct the tables
##here